In [2]:
# ensure classes imported from .py files are dynamically updated
%load_ext autoreload
%autoreload 2

# plot matplots nicely
%matplotlib inline  

In [3]:
import numpy as np
import pandas as pd
import datetime
import xarray as xr
from dateutil.relativedelta import relativedelta
from climate_drought import config, drought_indices as dri, utils

In [4]:
# configure inputs
aa = config.AnalysisArgs(latitude=35,longitude=-120,start_date='20200116',end_date='20200410')
cf = config.Config(outdir= 'output',indir='input',verbose=False)
#cdi = dri.CDI(cf,aa)

In [6]:
spi = dri.SPI(cf,aa)
spi.download()
spi.process()

2023-04-17  08:59:04 INFO Downloaded file 'output\precip_19850101-20221231_monthly_35_-120.nc' already exists.
2023-04-17  08:59:04 INFO Initiating processing of ERA5 daily data.
2023-04-17  08:59:05 INFO Input precipitation, 456 values: 0.000 0.012 
2023-04-17  08:59:05 INFO 

2023-04-17  08:59:06 DEBUG scaled values: 0.000 0.017
2023-04-17  08:59:07 DEBUG alphas: 0.949 3.479 betas: 0.000 0.003
2023-04-17  08:59:09 INFO SPI, 456 values: -3.081 3.090


,tp,spi
time,,
2020-02-01,0.000179,-0.983137
2020-03-01,0.003710,-0.540281
2020-04-01,0.002878,0.469652


In [8]:
sma_edo = dri.SMA_GDO(cf,aa)
sma_edo.download()
sma_edo.process()

2023-04-17  08:59:47 INFO Downloaded files available.
2023-04-17  08:59:49 INFO Completed processing of ERA5 soil water data.


,smant
2020-01-11,NaN
2020-01-21,-0.79
2020-02-01,-1.40
2020-02-11,-2.19
2020-02-21,-2.31
2020-03-01,-2.33
2020-03-11,-1.67
2020-03-21,-0.29
2020-04-01,0.83


In [10]:
fpr = dri.FPAR_EDO(cf,aa)
fpr.process()
fpr.process()

IndentationError: unexpected indent (202300053.py, line 2)

In [46]:
cdi = dri.CDI(cf,aa,spi=spi,sma=sma,fpr=fpr)

In [55]:
shift_date = lambda arr, n: np.pad(arr[:-n],(n,0),'constant',constant_values=(np.nan,))
idx_np = lambda df: df.to_numpy().flatten()

In [63]:
spi_cdi = idx_np(spi.data).repeat(3)
spi_tme = idx_np(spi.data.index).repeat(3)
spi_cdi = shift_date(spi_cdi,3)
spi_tme = shift_date(spi_tme,3)

In [72]:
spi.data.index[0] + datetime.timedelta(days=10)

Timestamp('2020-01-11 00:00:00')

In [53]:


        # ------ SPI ------ :
        df_spi = self.spi.process() if len(self.spi.data) == 0 else self.spi.data
        self.df_spi = df_spi

        # Convert to dekads for consistency with other timeseries
        # Use the same value 3 times a month for each dekad
        

        # Shift backwards by 3 dekads
        
        self.spi_np = spi
        
        # ------ SMA ------
        df_sma = self.sma.process() if len(self.sma.data) == 0 else self.sma.data

        # Shift backwards by 2 dekads
        sma = shift_date(idx_np(df_sma),2)
        self.sma_np = sma

array([   nan,  0.726,  0.722,  0.521,  0.344, -0.011, -0.188, -0.535,
       -0.55 , -0.588], dtype=float32)

In [52]:
cdi.spi_np

array([        nan,         nan,         nan, -0.02924945, -0.02924945,
       -0.02924945, -0.98313719, -0.98313719, -0.98313719, -0.54028095,
       -0.54028095, -0.54028095])

In [47]:
cdi.process()

2023-04-14  12:31:59 INFO Computing Combined Drought Indicator...
2023-04-14  12:32:01 INFO Completed processing of ERA5 soil water data.


ValueError: Length of values (12) does not match length of index (10)

In [67]:
df_spi = cdi.spi.process()
df_sma = cdi.sma.process()
df_fpr = cdi.fpr.process()


2023-04-05  13:22:58 INFO Initiating processing of ERA5 daily data.
2023-04-05  13:22:58 INFO Input precipitation, 456 values: 0.000 0.005 
2023-04-05  13:22:58 INFO 

2023-04-05  13:22:58 DEBUG scaled values: 0.002 0.012
2023-04-05  13:22:58 DEBUG alphas: 9.687 17.504 betas: 0.000 0.001
2023-04-05  13:22:58 INFO SPI, 456 values: -3.046 2.676
2023-04-05  13:23:00 INFO Completed processing of ERA5 soil water data.
2023-04-05  13:23:02 INFO Completed processing of ERA5 fAPAR data.


In [85]:
df_sma

,smant
time,
2020-01-01,0.54
2020-01-11,0.19
2020-01-21,-0.08
2020-02-01,-0.05
2020-02-11,-0.14
...,...
2022-09-21,-0.60
2022-10-01,-0.37
2022-10-11,-0.94


In [81]:
len(df_spi.spi.to_numpy().flatten().repeat(3))

108

2023-04-05  12:14:21 INFO Initiating processing of ERA5 daily data.
2023-04-05  12:14:21 INFO Input precipitation, 456 values: 0.000 0.005 
2023-04-05  12:14:21 INFO 

2023-04-05  12:14:21 DEBUG scaled values: 0.002 0.012
2023-04-05  12:14:21 DEBUG alphas: 9.687 17.504 betas: 0.000 0.001
2023-04-05  12:14:21 INFO SPI, 456 values: -3.046 2.676
2023-04-05  12:14:23 INFO Completed processing of ERA5 soil water data.
2023-04-05  12:14:24 INFO Completed processing of ERA5 fAPAR data.


In [58]:
cdi.fpr.filelist

['input/fpanv\\fpanv_m_wld_20120101_20121221_t.nc',
 'input/fpanv\\fpanv_m_wld_20130101_20131221_t.nc',
 'input/fpanv\\fpanv_m_wld_20140101_20141221_t.nc',
 'input/fpanv\\fpanv_m_wld_20150101_20151221_t.nc',
 'input/fpanv\\fpanv_m_wld_20160101_20161221_t.nc',
 'input/fpanv\\fpanv_m_wld_20170101_20171221_t.nc',
 'input/fpanv\\fpanv_m_wld_20180101_20181221_t.nc',
 'input/fpanv\\fpanv_m_wld_20190101_20191221_t.nc',
 'input/fpanv\\fpanv_m_wld_20200101_20201221_t.nc',
 'input/fpanv\\fpanv_m_wld_20210101_20211221_t.nc',
 'input/fpanv\\fpanv_m_wld_20220101_20221221_t.nc',
 'input/fpanv\\fpanv_m_wld_20230101_20230311_t.nc']

In [47]:
len(df_spi)

35

In [50]:
df_fpr

,fpanv
time,
2020-01-01,NaN
2020-01-11,NaN
2020-01-21,-1.371
2020-02-01,-1.078
2020-02-11,-0.787
...,...
2022-11-11,NaN
2022-11-21,NaN
2022-12-01,NaN


In [34]:
idx_np = lambda df: df.to_numpy().flatten()
shift_date = lambda arr, n: np.pad(arr[:-n],(n,0),'constant',constant_values=(np.nan,))

# use fAPAR times as CDI delivery dates
time = df_fpr.index

# fapar needs to use previous dekad's values
fpr = shift_date(idx_np(df_fpr),1)

# sma needs to use dekad -2 values
sma = shift_date(idx_np(df_sma),2)

# convert spi to dekads
spi = idx_np(df_spi.spi).repeat(3)[:-2]

# spi needs to be from the previous month
spi = shift_date(spi,3)

# spi is in months, and others are in dekads, so repeat spi 3 times before combining with others
#spi = spi.repeat(3)


In [39]:
df_spi.inerp

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [38]:
utils.to_dekads(df_spi)

,tp,spi
2020-01-01,0.001882,0.928141
2020-02-01,0.002690,1.106046
2020-03-01,0.000918,0.265101
2020-04-01,0.001049,-0.339632


,tp,spi
time,,
2020-01-01,0.001882,0.928141
2020-02-01,0.002690,1.106046
2020-03-01,0.000918,0.265101
2020-04-01,0.001049,-0.339632


In [16]:
arr=np.pad(df.to_numpy().flatten()[:-1],(1,0),'constant',constant_values=(np.nan,))

In [35]:
pd.DataFrame({'sdf':arr},index=time)

,sdf
time,
2020-01-01,NaN
2020-01-11,NaN
2020-01-21,NaN
2020-02-01,-1.371
2020-02-11,-1.078
2020-02-21,-0.787
2020-03-01,-1.231
2020-03-11,-0.999
2020-03-21,-0.859
